## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [23]:
import os
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from pyproj import CRS
from shapely.ops import nearest_points

os.environ['HTTP_PROXY'] = "http://10.1.10.99:8080"
os.environ['HTTPS_PROXY'] ="http://10.1.10.99:8080"

# Read the data (replace "None" with your own code)
path = "activity_locations.txt"

address = pd.read_csv(path, sep=';')

address

,id,addr
0,0,"Koirasaarentie 35, 00590 Helsinki"
1,1,"Gustaf Hällströmin katu 2, 00560 Helsinki"


In [24]:
# Geocode activity locations

# Geocode the addresses using Nominatim
addr_geo = geocode(address['addr'], provider='nominatim', user_agent='autogis_AB', timeout=4)
addr_geo

,geometry,address
0,POINT (25.02474 60.16878),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta..."
1,POINT (24.96289 60.20490),"Unicafe Physicum, 2, Gustaf Hällströmin katu, ..."


In [25]:
# Read shopping center data
# Read file from Shapefile
fp = "shopping_centers.shp"
shopping_centers = gpd.read_file(fp)

In [26]:
# Check coordinate reference system information (re-project/define if needed)

print(addr_geo.crs)
print(shopping_centers.crs)

EPSG:4326
epsg:3879


In [27]:
addr_geo = addr_geo.to_crs(CRS.from_epsg(3879))
print(addr_geo.crs)
print(shopping_centers.crs)

epsg:3879
epsg:3879


In [30]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:

addr_geo

,geometry,address
0,POINT (25501373.676 6672876.994),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta..."
1,POINT (25497942.152 6676901.898),"Unicafe Physicum, 2, Gustaf Hällströmin katu, ..."


In [31]:
shopping_centers

,address,id,name,addr,geometry
0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109)
1,"Salaattiasema, 14-20, Mannerheimintie, Kluuvi,...",1,Forum,"Mannerheimintie 14-20, 00100 Helsinki, Finland",POINT (25496573.542 6672878.360)
2,"Sports Academy, 11, Piispansilta, Matinkylä, S...",2,Iso-Omena,"Piispansilta 11, 02230 Espoo, Finland",POINT (25485431.705 6672252.372)
3,"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...",3,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland",POINT (25489491.076 6678322.265)
4,"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",4,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa",POINT (25497943.932 6686656.982)
5,"Yoga Valo, 5, Hermannin rantatie, Verkkosaari,...",5,REDI,"Hermannin rantatie 5, 00580 Helsinki",POINT (25498829.274 6674970.005)
6,"Pasilansilta, Keski-Pasila, Pasila, Keskinen s...",6,Tripla,"Pasilansilta 11, 00520 Helsinki",POINT (25496361.661 6676135.584)


Can you think of other application cases for the nearest neighbour analysis?

In [35]:
def get_nearest_values(row, other_gdf, geom = 'geometry', value = 'name'):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf[geom].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[geom], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf[geom] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value].values[0]
    
    return nearest_value

In [36]:
addr_geo["nearest_sc"] = addr_geo.apply(get_nearest_values, other_gdf=shopping_centers, geom="geometry", value="name", axis=1)

In [37]:
addr_geo

,geometry,address,nearest_sc
0,POINT (25501373.676 6672876.994),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta...",REDI
1,POINT (25497942.152 6676901.898),"Unicafe Physicum, 2, Gustaf Hällströmin katu, ...",Tripla
